# Runner example

This notebook will go through the options of the runner class. We will show how to fit and evaluate a model in parallel, and how to do cross-validation. 

We will also demonstrate how to transfer and extend a HBR model using the runner.


### Imports

In [1]:
import pandas as pd
import os
from pcntoolkit.dataio.norm_data import NormData
from pcntoolkit.normative_model.norm_conf import NormConf
from pcntoolkit.normative_model.norm_blr import NormBLR
from pcntoolkit.regression_model.blr.blr_conf import BLRConf
from pcntoolkit.normative_model.norm_hbr import NormHBR
from pcntoolkit.regression_model.hbr.hbr_conf import HBRConf

from pcntoolkit.util.runner import Runner
from pcntoolkit.util.plotter import plot_centiles, plot_qq

os.makedirs("resources/data", exist_ok=True)

# Load data

First we download a small example dataset from github. Saving this dataset on your local device (under 'resources/data/fcon1000.csv' for example) saves time and bandwidth if you re-run this notebook.

In [2]:
# If you are running this notebook for the first time, you need to download the dataset from github.
# If you have already downloaded the dataset, you can comment out the following line

pd.read_csv(
    "https://raw.githubusercontent.com/predictive-clinical-neuroscience/PCNtoolkit-demo/refs/heads/main/data/fcon1000.csv"
).to_csv("resources/data/fcon1000.csv", index=False)
data = pd.read_csv("resources/data/fcon1000.csv")
covariates = ["age"]
batch_effects = ["sex", "site"]
response_vars = "lh_G_cingul-Post-dorsal_thickness,lh_G_cingul-Post-ventral_thickness,lh_G_cuneus_thickness,lh_G_front_inf-Opercular_thickness,lh_G_front_inf-Orbital_thickness,lh_G_front_inf-Triangul_thickness,lh_G_front_middle_thickness,lh_G_front_sup_thickness,lh_G_Ins_lg&S_cent_ins_thickness,lh_G_insular_short_thickness,lh_G_occipital_middle_thickness,lh_G_occipital_sup_thickness,lh_G_oc-temp_lat-fusifor_thickness,lh_G_oc-temp_med-Lingual_thickness,lh_G_oc-temp_med-Parahip_thickness,lh_G_orbital_thickness,lh_G_pariet_inf-Angular_thickness,lh_G_pariet_inf-Supramar_thickness,lh_G_parietal_sup_thickness,lh_G_postcentral_thickness,lh_G_precentral_thickness,lh_G_precuneus_thickness,lh_G_rectus_thickness,lh_G_subcallosal_thickness,lh_G_temp_sup-G_T_transv_thickness,lh_G_temp_sup-Lateral_thickness,lh_G_temp_sup-Plan_polar_thickness,lh_G_temp_sup-Plan_tempo_thickness,lh_G_temporal_inf_thickness,lh_G_temporal_middle_thickness,lh_Lat_Fis-ant-Horizont_thickness,lh_Lat_Fis-ant-Vertical_thickness,lh_Lat_Fis-post_thickness,lh_Pole_occipital_thickness,lh_Pole_temporal_thickness,lh_S_calcarine_thickness".split(",")
norm_data = NormData.from_dataframe(
    name="full",
    dataframe=data,
    covariates=["age"],
    batch_effects=["sex", "site"],
    response_vars=response_vars,
)

# Leave two sites out for doing transfer and extend later
transfer_sites = ["Milwaukee_b", "Oulu"]
transfer_data, fit_data = norm_data.split_batch_effects({"site": transfer_sites}, names=("transfer", "fit"))

# Split into train and test sets
train, test = fit_data.train_test_split()
transfer_train, transfer_test = transfer_data.train_test_split()

## Configure the normative model

The normative model will be configured using a `NormConf` object, containing save and log paths and the preprocessing configurations, and a `RegConf` object, specific to the regression model type. 

Our `NormConf` object configures:
- a save path paths and whether to save the model and results
- a standardization step for both the covariates (inscaler) and the response vars (outscaler)
- a Bspline basis expansion of order 3 with 5 knots

In [3]:
# Create a NormConf object
norm_conf = NormConf(
    savemodel=True,
    saveresults=True,
    save_dir="resources/blr/save_dir",
    inscaler="standardize",
    outscaler="standardize",
    basis_function="bspline",
    basis_function_kwargs={"order": 3, "nknots": 5},
)

Process: 2010482 - Configuration of normative model is valid.


## Configure the regression model


In [4]:
blr_conf = BLRConf(
    optimizer="l-bfgs-b",
    n_iter=200,
    heteroskedastic=True,
    intercept=True,
    fixed_effect=False,
    fixed_effect_var=False,
    warp="WarpSinhArcsinh",
    warp_reparam=True,
)

Process: 2010482 - Configuration of regression model is valid.


## Combine normative and blr conf in normative model
We can either use the NormBLR constructor, or the factory method to create a normative BLR model

In [5]:
# Using the constructor
new_model = NormBLR(norm_conf=norm_conf, reg_conf=blr_conf)

## Fit the model
Normally we would just call 'fit_predict' on the model directly, but because we want to use the runner to do cross-validation in parallel, we need to first create a runner object. 

In [6]:
runner = Runner(
    cross_validate=False,
    cv_folds=3,
    parallelize=True,
    environment="/project/3022000.05/projects/stijdboe/envs/dev_refactor",
    job_type="slurm",  # or "slurm" if you are on a slurm cluster
    n_jobs=10,
    log_dir="resources/runner_output/log_dir",
    temp_dir="resources/runner_output/temp_dir",
)
# Local parallelization might not work on login nodes due to resource restrictions

The runner object will now fit the model in parallel, and save the results in save directories that it will create for each fold.

In [ ]:
runner.fit_predict(new_model, train, test)


-----------------------------------------------------
            PCNtoolkit Job Status Monitor ®
-----------------------------------------------------
Job ID      Name      State      Time      Nodes
-----------------------------------------------------

46821561    job_0     PENDING    0:00                    
46821562    job_1     PENDING    0:00                    
46821563    job_2     PENDING    0:00                    
46821564    job_3     PENDING    0:00                    
46821565    job_4     PENDING    0:00                    
46821566    job_5     PENDING    0:00                    
46821567    job_6     PENDING    0:00                    
46821568    job_7     PENDING    0:00                    
46821569    job_8     PENDING    0:00                    
46821570    job_9     PENDING    0:00                    

-----------------------------------------------------
Total active jobs: 10
Total completed jobs: 0
Total failed jobs: 0
-----------------------------------------

### Loading a fold model
We can load a model for a specific fold by calling `load_model` on the runner object. This will return a `NormBLR` object, which we can inspect and use to predict on new data.


In [8]:
fitted_model = runner.load_model()
display(fitted_model)

Process: 1982211 - Configuration of normative model is valid.
Process: 1982211 - Configuration of normative model is valid.
Process: 1982211 - Configuration of regression model is valid.
Process: 1982211 - Configuration of regression model is valid.
Process: 1982211 - Configuration of regression model is valid.
Process: 1982211 - Configuration of regression model is valid.


## Inspecting the model 

The norm_blr model contains a collection of regression models, one for each response variable. We can inspect those models individually by calling `norm_blr.regression_models.get("{responsevar}")`

In [9]:
single_hbr_model = fitted_model.regression_models.get("rh_MeanThickness_thickness")  # type: ignore
single_hbr_model.__dict__

{'_name': 'rh_MeanThickness_thickness',
 '_reg_conf': BLRConf(n_iter=200, tol=1e-05, ard=False, optimizer='l-bfgs-b', l_bfgs_b_l=0.1, l_bfgs_b_epsilon=0.1, l_bfgs_b_norm='l2', intercept=True, fixed_effect=False, heteroskedastic=True, intercept_var=False, fixed_effect_var=False, warp='WarpSinhArcsinh', warp_reparam=True),
 'is_fitted': True,
 '_is_from_dict': True,
 'hyp': array([-0.00271425,  0.40459816,  0.57171555,  0.14023606, -0.02021072,
        -0.02146683, -0.00490682, -0.28481558, -0.09296211, -0.06571992,
        -0.00115522,  0.00524956,  0.00339416,  0.00250672, -0.03653256,
        -0.00530122,  0.00506945]),
 'nlZ': 979.881210875374,
 'N': 744,
 'D': 8,
 'lambda_n_vec': None,
 'Sigma_a': array([[1.06792757, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 1.00115589, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.9947642 , 0.        , 0.  

## Evaluation
Calling `predict` will extend the predict_data object with a number of useful arrays.
1. `measures`: DataArray, which contains a number of evaluation statistics. 
1. `zscores`: the predicted z-scores for each datapoint.  
1. `centiles`: the predicted centiles of variation evaluated at each covariate in the dataset. 


In [10]:
runner.predict(new_model, test)
# display(test.measures.to_pandas().T.round(4))


--------------------------------------------------------
             PCNtoolkit Job Status Monitor ®
--------------------------------------------------------
Job ID      Name              State      Time      Nodes
--------------------------------------------------------

46821015    job_1             FAILED                        
46821014    job_0             FAILED                        

--------------------------------------------------------    
All jobs completed!
--------------------------------------------------------



Datasets with a zscores DataArray will have the `.plot_qq()` function available:

In [ ]:
display(test.zscores.to_pandas())  # the zscores

In [ ]:
display(test.centiles.to_dataframe().unstack(level=["response_vars", "cdf"]))

In [ ]:
plot_qq(test)

And `plot_centiles()` can be called as a function of the model. A synthetic dataset is created internally, so we need to pass the original dataset (`train` in this case) as a template. We also need to pass which covariate is to be plotted on the x-axis, and the batch-effects for which the centiles are to be plotted. 

The lines correspond to the CDF values of: [0.05, 0.25, 0.5, 0.75, 0.95]. It is also possible to pass a list of CDF values to plot.

It may seem strange that the centiles do not match the plotted data, but that is because the centiles are calculated for a single batch effect, and it is superimposed on the full dataset. The blue markers correspond to the data for which the centiles are calculated. 

In [ ]:
plot_centiles(
    fitted_model,
    train,
    covariate="age",
    show_data=True,
    hue_data="sex",
    markers_data="site",
)

The values of 0.1587 and 0.8413 correspond to a standard deviation of -1 and 1. We plot the centiles again for these values, and we also highlight a specific site. 


In [ ]:
plot_centiles(
    fitted_model,
    train,
    covariate="age",
    centiles=[0.1587, 0.8413],
    show_data=True,
    batch_effects={"site": ["Beijing_Zang"]},
)

# Transfering with the runner

The runner can also be used to transfer or extenda model to a new dataset. This is done by calling `transfer` on the runner object. This will transfer the model to the new dataset, and save the transfered model in the save directory.

runner.transfer(transfer_train, transfer_test)

Let's first fit an HBR model, and then transfer it to the transfer dataset.



In [ ]:
from pcntoolkit.regression_model.hbr.prior import make_prior


mu = make_prior(
    name="mu",
    linear=True,
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 5.0)),
    intercept=make_prior(
        random=True,
        sigma=make_prior(dist_name="HalfNormal", dist_params=(1.0,)),
        mu=make_prior(dist_name="Normal", dist_params=(0.0, 0.5)),
    ),
)
sigma = make_prior(
    name="sigma",
    linear=True,
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 3.0)),
    intercept=make_prior(
        dist_name="Normal",
        dist_params=(
            1.0,
            1.0,
        ),
    ),
    mapping="softplus",
    mapping_params=(0.0, 3.0),
)

# Configure the HBRConf object
hbr_conf = HBRConf(
    draws=2048,
    tune=512,
    chains=4,
    pymc_cores=16,
    likelihood="Normal",
    mu=mu,
    sigma=sigma,
    nuts_sampler="nutpie",
)
save_dir = "resources/hbr_transfer_runner/save_dir"

norm_conf = NormConf(
    savemodel=True,
    saveresults=True,
    save_dir="resources/hbr_transfer_runner/save_dir",
    inscaler="standardize",
    outscaler="standardize",
    basis_function="bspline",
    basis_function_kwargs={"order": 3, "nknots": 5},
)

new_hbr_model = NormHBR(norm_conf=norm_conf, reg_conf=hbr_conf)
runner = Runner(
    cross_validate=False,
    cv_folds=3,
    parallelize=True,
    time_limit="00:15:00",
    job_type="slurm",  # or "slurm" if you are on a slurm cluster
    n_jobs=2,
    log_dir="resources/hbr_transfer_runner/log_dir",
    temp_dir="resources/hbr_transfer_runner/temp_dir",
)
runner.fit_predict(new_hbr_model, train, test)

In [ ]:
norm_hbr = NormHBR.load("resources/hbr_transfer_runner/save_dir")

In [ ]:
norm_hbr.predict(test)
try:
    norm_hbr.predict(transfer_test)
except Exception as e:
    print(
        f"""
≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠
The original model cannot be used to predict on the transfer dataset, because it was not trained on it

See the useful error message below:

{e}
≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠
"""
    )

In [ ]:
to_transfer = NormHBR.load("resources/hbr_transfer_runner/save_dir")
runner = Runner(
    cross_validate=False,
    parallelize=True,
    job_type="local",  # or "slurm" if you are on a slurm cluster
    n_jobs=2,
    log_dir="resources/hbr_transfer_runner/log_dir",
    temp_dir="resources/hbr_transfer_runner/temp_dir",
)
runner.transfer_predict(to_transfer, transfer_train, transfer_test)

In [ ]:
# transfered_model = NormHBR.load("resources/hbr_transfer_runner/save_dir_transfer")
transfered_model = runner.load_fold_model(0)
transfered_model.predict(transfer_test)
try:
    transfered_model.predict(test)
except Exception as e:
    print(
        f"""
≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠
The transfered model cannot be used to predict on the original dataset, because it was transfered to a different dataset

See the useful error message below:

{e}
≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠≠
"""
    )

In [ ]:
to_extend = NormHBR.load("resources/hbr_transfer_runner/save_dir")
runner.extend(to_extend, transfer_train)

In [ ]:
extended_model = runner.load_fold_model(0)
extended_model.predict(transfer_test)
extended_model.predict(test)
print("The extended model can now be used to predict on both the original and transfer dataset.")

And that's it, now you have seen how to:
- Use the runner to do cross-validation in parallel
- Inspect the model of a specific fold
- Evaluate the model on a test set
- Create useful plots

We hope this tutorial was useful. If you have any questions or remarks, please let us know on GitHub. Thanks!
